# Installation

In [11]:
# %pip install torch
# %pip install torchtext

In [12]:
!unzip colab.zip

Archive:  colab.zip
   creating: data/
  inflating: data/test.csv           
  inflating: data/whole.csv          
  inflating: data/train.csv          
  inflating: data/val.csv            
  inflating: svd_96000_3_100.pt      
  inflating: trainClassVocab12000.pth  
  inflating: trainClassVocab24000.pth  
  inflating: trainClassVocab60000.pth  
  inflating: trainClassVocab72000.pth  
  inflating: trainClassVocab96000.pth  
  inflating: trainWordVocab12000.pth  
  inflating: trainWordVocab24000.pth  
  inflating: trainWordVocab60000.pth  
  inflating: trainWordVocab72000.pth  
  inflating: trainWordVocab96000.pth  


# Import

In [13]:
import torch
torch.manual_seed(42) # Set Seed for reproducibility
import csv
import numpy as np
import pandas as pd
from nltk import RegexpTokenizer
import re
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader, Dataset
from numpy.linalg import norm
import scipy as sp

In [14]:
sizeOfTrainData = int(0.2*120000)

# Data

In [15]:
def getDescriptions(csvFilePath, numOfRows):
    df = pd.read_csv(csvFilePath, nrows=numOfRows, usecols=['Description'])
    # print(df.shape)
    # df.head()
    textData = df['Description']
    return textData

# Dataset

In [16]:
UNKNOWN_TOKEN = "<UNK>"
NUMBER_TOKEN = "<NUM>"
START_TOKEN = "<START>"
END_TOKEN = "<END>"
PAD_TOKEN = "<PAD>"
class embedDataset(Dataset):
    def __init__(self, data : list[str], wordVocab:Vocab|None = None):
        self.data = data
        self.wordVocab = wordVocab
        # if wordVocab is None:
        #     print("wordVocab needs to be provided")
        #     exit(1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        des =  self.tokenize(self.data[idx])
        if self.wordVocab is not None:
            return torch.tensor(self.wordVocab.lookup_indices(des), dtype=torch.long)

    def tokenize(self, des:str):
        des = des.lower()
        tokDes = RegexpTokenizer(r'\w+').tokenize(des)
        # add start and end token
        tokDes = [START_TOKEN] + tokDes + [END_TOKEN]
        # replace numbers with <NUM>
        tokDes = [re.sub(r'\d+', NUMBER_TOKEN, word) for word in tokDes]
        return tokDes


# word2vec using SVD

In [17]:
class SVDword2vec(torch.nn.Module):
    def __init__(self, dataset: embedDataset, windowSize: int, wordVocab: Vocab, embeddingSize:int|None = None):
        super().__init__()
        self.dataset = dataset
        self.windowSize = windowSize
        self.embeddingSize = embeddingSize
        self.wordVocab = wordVocab
        self.coOccurrenceMatrix = np.zeros((len(self.wordVocab), len(self.wordVocab)))
        self.minVariance = 0.95
        self.vectors = None
        self.minEmbeddingSize = 100


    def createCoOccurrenceMatrix(self):
        for i in range(len(self.dataset)):
            for j in range(len(self.dataset[i])):
                window = self.dataset[i][max(0, j - self.windowSize):min(len(self.dataset[i]), j + self.windowSize + 1)]
                word = self.dataset[i][j]
                for k in range(len(window)):
                    if window[k] != word:
                        self.coOccurrenceMatrix[word.item(), window[k].item()] += 1
        return

    def train(self):
        self.createCoOccurrenceMatrix()
        U, S, Vt = sp.sparse.linalg.svds(self.coOccurrenceMatrix)
        den = sum(S)
        c_sum = 0
        if self.embeddingSize is None:
            for i, x in enumerate(S):
                c_sum = c_sum + x
                if (c_sum/den > self.minVariance):
                    self.embeddingSize = i+1
                    break
        if self.embeddingSize < self.minEmbeddingSize :
          self.embeddingSize = self.minEmbeddingSize
        self.vectors = U[:, :self.embeddingSize]
        self.vectors = np.array([v/norm(v) for v in self.vectors])
        return

# load Vocab

In [18]:
wordVocab = torch.load("trainWordVocab"+str(sizeOfTrainData)+".pth")
print("Vocab size: ", len(wordVocab))

Vocab size:  19369


# train dataset

In [19]:
trainData = getDescriptions('./data/train.csv', sizeOfTrainData)
print(trainData[:5])

0    London - The British Broadcasting Corporation,...
1    Embattled insurance broker #39;s banks agree t...
2    AP - Derek Jeter turned a season that started ...
3    When the Genesis capsule comes back to Earth w...
4     NEW YORK (Reuters) - U.S. stocks were set to ...
Name: Description, dtype: object


In [20]:
trainDataset = embedDataset(trainData, wordVocab)
print(trainDataset[6])

tensor([   2,   76,  101, 1888,   61,   16,  127,   50, 3777, 1426, 2432, 3842,
         101,   13,   18,    0,  512,  171,    9,   25,  732,    7, 2062,    5,
         862, 6746,   21,   20,  642,    7, 3173,   50,  833,    3])


# Train

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [22]:
# def saveSVDword2vec(model: SVDword2vec):
#     modelPath = f"svd_{sizeOfTrainData}_{model.windowSize}_{model.embeddingSize}.pt"
#     torch.save(model, modelPath)
#     print(f"Model saved at {modelPath}")
#     return

In [23]:
windowSize = 3
# trainDataset = trainDataset.to(device)
mySVDword2vec = SVDword2vec(trainDataset,windowSize,wordVocab).to(device)
mySVDword2vec.train()
print("model training done")
# mySVDword2vec.saveVectors()
# saveSVDword2vec(mySVDword2vec)
vectorsPath = f"svd_{sizeOfTrainData}_{mySVDword2vec.windowSize}_{mySVDword2vec.embeddingSize}.pt"
torch.save(mySVDword2vec.vectors, vectorsPath)
print(f"vectors saved at {vectorsPath}")
print("vectors shape", mySVDword2vec.vectors.shape)

model training done
vectors saved at svd_24000_3_100.pt
vectors shape (19369, 6)
